In [2]:
import tweepy
from dotenv import load_dotenv
import os
import sqlalchemy as db
import psycopg2
import datetime
import pandas as pd
import numpy as np

In [91]:
load_dotenv()
database_username= os.getenv('DATABASE_USERNAME')
database_password= os.getenv('DATABASE_PASSWORD')
database_ip= os.getenv('DATABASE_IP')
database_name=os.getenv('DATABASE_NAME')

In [92]:
def getdata():
    database_conection=db.create_engine(f"postgresql://{database_username}:{database_password}@{database_ip}/{database_name}")
    connection = database_conection.raw_connection()
    metadata=db.MetaData()
    cursor = connection.cursor()
    df = pd.read_sql_query('SELECT p.nombre, t.id, t.id_terremotos, t.fecha, t.clasificacion, t.id_ciudad, t.distancia FROM public.poblacion_chile as p JOIN public.twitter as t ON t.id_ciudad = p.id WHERE fecha is null',con=database_conection)
    connection.close()
    df1 = df[df['clasificacion'] == 1]
    df2 = df[df['clasificacion'] == 2]
    df3 = df[df['clasificacion'] == 3]
    return df1, df2, df3

In [93]:
df1, df2, df3 =  getdata()

In [94]:
df1

,nombre,id,id_terremotos,fecha,clasificacion,id_ciudad,distancia
0,Alto del Carmen,3,us7000isxn,None,1,25,231.3


In [58]:
#usamos dotenv para traer las claves del .env y mantener seguras las credenciales
load_dotenv()

API_KEY =os.getenv('API_KEY')
API_SECRET =os.getenv('API_SECRET')
ACCESS_TOKEN= os.getenv('ACCESS_TOKEN')
ACCESS_SECRET=os.getenv('ACCESS_SECRET')

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
 
api = tweepy.API(auth, wait_on_rate_limit=True)
 
 
def ObtenerUsuario(user_name):
    return api.get_user(screen_name=user_name)
 
def PublicarTweet(message):
    api.update_status(status=message)
 
 
def ObtenerTweetsDeUsuario(user_name, number_of_tweets):
    tweets = api.user_timeline(screen_name=user_name, count=number_of_tweets)
    return tweets
 
 
def ObtenerSeguidoresDeUsuario(user_name, number_of_followers):
    return tweepy.Cursor(method=api.get_followers, screen_name=user_name).items(number_of_followers)
 
 
#def ObtenerNumeroDeSeguidoresDeUsuario(user_name):
#    return api.get_user(screen_name=user_name).followers_count
 
 
#def ObtenerTweetDeUsuarioPorNumero(user_name, TweetNumber):
#    lastTweet = TweetNumber - 1
#    return api.user_timeline(screen_name=user_name, count=TweetNumber)[lastTweet]
 
 
#def RetwittearTweet(tweet):
#    try:
#        api.retweet(id=tweet.id)
#    except:
# 
#        print("Ya has retwitteado este tweet")


In [ ]:
alert = '\U00002757'
auto = '\U0001F697'
no = '\U0001F6AB'
caminar = '\U0001F6B6'
df2 =  getdata()[1]
for i in range(len(df2)):
    ciudad = (df2.iloc[i]['nombre'])
    dist = (df2.iloc[i]['distancia'])
    alerta_naranja = f'''{alert} Quedate donde estás. Sismo medio a {dist} km de {ciudad}.
    El manejo de vehículos {auto} no es seguro {no} y podés tener dificultad de mantenerte en pie{caminar}. 
    Si estás en un vehiculo, detente. Agáchate, Cubrite y agarrate. Alejarse de muebles y ventanas. 
    Tranquilo, ya pasará.'''
    PublicarTweet(alerta_naranja)  

In [86]:
def updatetime():
    # This query fetches the 20 latest id from database
    database_conection=db.create_engine(f"postgresql://{database_username}:{database_password}@{database_ip}/{database_name}")
    connection = database_conection.raw_connection()
    metadata=db.MetaData()
    cursor = connection.cursor()
    sql_query = """
    UPDATE public.twitter
    SET fecha = current_timestamp
    WHERE fecha is null;
    """

    database_conection.execute(sql_query)
    #print('fetching ids from database')
    connection.close()

In [87]:
updatetime()